In [1]:
from bs4 import BeautifulSoup # html을 soup에 넣고 특정 class or data를 가져오는 library
import re
from urllib.parse import urlencode
from urllib.request import Request, urlopen
import requests
from selenium import webdriver
import os
import time
import shutil

In [2]:
def link_return(text):
    url = "http://www.dbdbdeep.com/ma2/inc/file_down.php?"
    p = re.compile('p_seq=.*"')
    value = p.search(str(text)).group()
    value = re.sub('amp;','',value)
    return url + value[:-1]

In [4]:
# dbdbdeep 사이트 접속
driver = webdriver.Chrome('./chromedriver')
# driver = webdriver.PhantomJS('/Users/yonggeoljo/phantomjs-2.1.1-macosx/bin/phantomjs')
driver.implicitly_wait(3)

In [5]:
driver.get("http://www.dbdbdeep.com/ma2/post/posting_main.php")
driver.find_element_by_class_name('margin_right10').click()

In [6]:
# 2중 로그인창 접속

window_before = driver.window_handles[0]
window_after = driver.window_handles[1]
driver.switch_to_window(window_after)

ID = 'yonggeol_test'
PWD = 'ghazlvk1'
e = driver.find_element_by_id("id")
e.clear()
e.send_keys(ID)
e = driver.find_element_by_id("pw")
e.clear()
e.send_keys(PWD)
driver.find_element_by_class_name('btn_global').click()

In [7]:
# 원래 페이지로 돌아와서 포스팅 클릭
driver.switch_to_window(window_before)
driver.find_element_by_css_selector("body > div.sub_BackgroundIMG > div.sub_MenuArea > ul > a:nth-child(1) > li").click()

In [7]:
with open('dbdbdeep.txt', 'w',encoding='utf-8') as f:
    f.write("start\n")

In [9]:
dir = '/Users/yonggeoljo/workspace/Graduate_school/Research/Credibility'
os.chdir(dir)

In [ ]:
#for문 하단 page 클릭
for page in range(0,len(driver.find_elements_by_class_name("sub_PageBTN"))):
    number = driver.find_elements_by_class_name("sub_PageBTN")
    number[page].click()
    time.sleep(1)
    
    # for문 게시글 클릭
    
    click_elements = driver.find_elements_by_class_name("Table_title")
    for i in range(0,len(click_elements)):
        click_elements = driver.find_elements_by_class_name("Table_title")
        click_elements[i].click()
        
        # 제목 Parsing
        title = driver.find_element_by_class_name("sub_detail_title")
        title = re,sub(",","_",title.text)
        
        # 내용 Parsing
        box = driver.find_element_by_class_name("sub_detail_textArea")
        bs = BeautifulSoup(box.get_attribute('innerHTML'), 'html.parser')
        subject = bs.find_all('th',{'class':'sub_detal_text'})
        subject = [i.text for i in subject]
        subject_content = bs.find_all('th',{'class':'sub_detal_text1'})
        subject_content = [i.text for i in subject_content]

        subject_content =list(map(str.strip,subject_content))
        numeric_value = [''.join(filter(lambda x: x.isdigit(), i)) for i in subject_content[1].split(",")]
        subject_content[3:4] = subject_content[3].split('/')
        subject_content[1:2] = numeric_value
        subject_content[0] = subject_content[0].split(",")[0] + '000원'
        subject_content.insert(0,title)
        subject_content = list(map(str.strip,subject_content))

        with open('dbdbdeep.txt', 'a',encoding='utf-8') as f:
            f.write(','.join(subject_content) +'\n')

            
        # 폴더 만들기
        folder_name = subject_content[6] + "_"+ title
        folder_name = re.sub('[=.#/?:$}]', '', folder_name)
        os.chdir("Source")
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        os.chdir(folder_name)

        # Html 저장
        title_box = driver.find_element_by_class_name("sub_detail_textArea").get_attribute('innerHTML')
        specific_context = driver.find_element_by_class_name("sub_detail_substance_area").get_attribute('innerHTML')
        Html = title_box + specific_context
        with open(folder_name+'.html', 'w',encoding='utf-8') as f:
            f.write(Html)


        #첨부파일 다운로드
        downloads = driver.find_elements_by_css_selector("body > div.sub_ContentArea > div.sub_center_area > form > div.sub_detail_substance_area > div:nth-child(10) > table > tbody > tr > th:nth-child(2)")
        if len(downloads) != 0: 
            for i in downloads:
                i.click()
                window_before = driver.window_handles[0]
                window_after = driver.window_handles[1]
                driver.switch_to_window(window_after)
                a_links = driver.find_elements_by_tag_name('a')
                for i in a_links:
                    try:
                        i.click()
                        time.sleep(5)
                    except:
                        pass
                time.sleep(3)
                driver.close()
                driver.switch_to_window(window_before)
            os.chdir(dir)
            download_list = os.listdir("/Users/yonggeoljo/Downloads")
            download_path = "/Users/yonggeoljo/Downloads"

            for i in download_list:
                try:
                    shutil.move(os.path.join(download_path,i),os.path.join(os.getcwd(),"Source",folder_name))
                except:
                    pass
            #클릭 다시
            driver.back()
            driver.refresh()
            time.sleep(2)
            number = driver.find_elements_by_class_name("sub_PageBTN")
            number[page].click()
        else:
            #클릭 다시
            os.chdir(dir)
            driver.back()
            driver.refresh()
            time.sleep(2)
            os.chdir(dir)
            number = driver.find_elements_by_class_name("sub_PageBTN")
            number[page].click()
driver.close()